In [ ]:
%tensorflow_version 2.x
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
tf.__version__


'2.8.2'

# Create one Dense layer

In [ ]:
# We've received DenseLayer().weights, trainable_weights, non_trainable_weights
class DenseLayer(tf.keras.layers.Layer):
    def __init__(self, units=1):
        print("init")
        super().__init__()
        # Neurons number 
        self.units = units

    # Only one time and before call
    def build(self, input_shape):
        print("build")
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer="random_normal",
                                 trainable=True)
        self.b = self.add_weight(shape=(self.units,),
                                 initializer="zeros",
                                 trainable=True)
    
    # After built
    def call(self, inputs):
        print("call")
        # Add own loss function
        regular = tf.reduce_mean(tf.square(self.w))
        self.add_loss(regular)
        # Add own metric function
        self.add_metric(regular, name="mean square weights")
        return tf.matmul(inputs, self.w) + self.b

print("prev init")
layer1 = DenseLayer(10)

print("prev call")
# Shape = [batch_size, data]
y = layer1(tf.constant([[1., 2., 3.]])) # Build, call
y = layer1(tf.constant([[1., 2., 3.]])) # Call
y = layer1(tf.constant([[1., 2., 3.]])) # Call
print(y)

prev init
init
prev call
build
call
call
call
tf.Tensor(
[[ 0.04388597 -0.05026485 -0.02006444  0.02260087 -0.05419174  0.3088227
  -0.00963814 -0.02691636 -0.2214514   0.19981988]], shape=(1, 10), dtype=float32)


# Create model

In [ ]:
class NeuralNetwork(tf.keras.Model):
    def __init__(self):
        super().__init__()
        # Hidden layer
        self.layer_1 = DenseLayer(128)
        # Output layer
        self.layer_2 = DenseLayer(10)

    def call(self, inputs):
        x = self.layer_1(inputs)
        x = tf.nn.relu(x)
        x = self.layer_2(x)
        x = tf.nn.softmax(x)
        return x


In [ ]:
model = NeuralNetwork()
model.compile(optimizer="Adam", # Default params
              loss="categorical_crossentropy", # Default params
              metrics=["accuracy"])
# Equal to:
# model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.001), # Default params
#               loss=tf.losses.categorical_crossentropy, # Default params
#               metrics=["accuracy"])

init
init


# Preparing data

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train / 255
x_test = x_test / 255

x_train = tf.reshape(tf.cast(x_train, tf.float32), [-1, 28 * 28])
x_test = tf.reshape(tf.cast(x_test, tf.float32), [-1, 28 * 28])

y_train = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)

# Fit model

In [ ]:
# Loss = categorical_enthropy + regular
# Two metrict: acc and our own
model.fit(x_train, y_train, batch_size=32, epochs=5)


Epoch 1/5
build
call
build
call
call
call
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3122 - accuracy: 0.9154 - mean square weights: 0.0098
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1570 - accuracy: 0.9613 - mean square weights: 0.0159
Epoch 3/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1211 - accuracy: 0.9725 - mean square weights: 0.0191
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1050 - accuracy: 0.9774 - mean square weights: 0.0206
Epoch 5/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0940 - accuracy: 0.9817 - mean square weights: 0.0211


In [ ]:
print(model.evaluate(x_test, y_test_cat))

call
call
313/313 [==============================] - 1s 3ms/step - loss: 0.1093 - accuracy: 0.9763 - mean square weights: 0.0211
[0.10931607335805893, 0.9763000011444092, 0.021097995340824127, 0.021097995340824127]
